In [1]:
# to handle datasets
import pandas as pd
import numpy as np

import seaborn as sns

# for plotting
import matplotlib.pyplot as plt

# for the yeo-johnson transformation
import scipy.stats as stats

# to divide train and test set
from sklearn.model_selection import train_test_split

# feature scaling
from sklearn.preprocessing import MinMaxScaler

# to save the trained scaler class
import joblib

# to visualise al the columns in the dataframe
pd.pandas.set_option('display.max_columns', None)

In [2]:
# load dataset
data = pd.read_csv('/home/jupyter/dataset/cali_ces.csv')

# rows and columns of the data
print(data.shape)

print(data.columns)

# visualize the dataset
data.head()

(20640, 10)
Index(['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income',
       'median_ces', 'ocean_proximity'],
      dtype='object')


longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0    -122.23     37.88                  41          880           129.0   
1    -122.22     37.86                  21         7099          1106.0   
2    -122.24     37.85                  52         1467           190.0   
3    -122.25     37.85                  52         1274           235.0   
4    -122.25     37.85                  52         1627           280.0   

   population  households  median_income  median_ces ocean_proximity  
0         322         126         8.3252      452600        NEAR BAY  
1        2401        1138         8.3014      358500        NEAR BAY  
2         496         177         7.2574      352100        NEAR BAY  
3         558         219         5.6431      341300        NEAR BAY  
4         565         259         3.8462      342200        NEAR BAY

In [3]:
# Perform feature engineering
# Create a new feature "total_rooms_per_person"
data['total_rooms_per_person'] = data['total_rooms']/data['population']
# Create a new feature "bedrooms_per_room"
data['bedrooms_per_room'] = data['total_bedrooms']/data['total_rooms']
# Create a new feature "income_per_person"
data['income_per_person'] = data['median_income']/data['population']
# Encode the categorical feature "ocean_proximity"
one_hot_encoded = pd.get_dummies(data.ocean_proximity,prefix='ocean_proximity')
one_hot_encoded = one_hot_encoded.astype(int)
data = pd.concat([data, one_hot_encoded], axis=1)
data.drop(['ocean_proximity'], axis=1)
data

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0        -122.23     37.88                  41          880           129.0   
1        -122.22     37.86                  21         7099          1106.0   
2        -122.24     37.85                  52         1467           190.0   
3        -122.25     37.85                  52         1274           235.0   
4        -122.25     37.85                  52         1627           280.0   
...          ...       ...                 ...          ...             ...   
20635    -121.09     39.48                  25         1665           374.0   
20636    -121.21     39.49                  18          697           150.0   
20637    -121.22     39.43                  17         2254           485.0   
20638    -121.32     39.43                  18         1860           409.0   
20639    -121.24     39.37                  16         2785           616.0   

       population  households  median_income  median_ces ocean_proximity  \
0             322         126         8.3252      452600        NEAR BAY   
1            2401        1138         8.3014      358500        NEAR BAY   
2             496         177         7.2574      352100        NEAR BAY   
3             558         219         5.6431      341300        NEAR BAY   
4             565         259         3.8462      342200        NEAR BAY   
...           ...         ...            ...         ...             ...   
20635         845         330         1.5603       78100          INLAND   
20636         356         114         2.5568       77100          INLAND   
20637        1007         433         1.7000       92300          INLAND   
20638         741         349         1.8672       84700          INLAND   
20639        1387         530         2.3886       89400          INLAND   

       total_rooms_per_person  bedrooms_per_room  income_per_person  \
0                    2.732919           0.146591           0.025855   
1                    2.956685           0.155797           0.003457   
2                    2.957661           0.129516           0.014632   
3                    2.283154           0.184458           0.010113   
4                    2.879646           0.172096           0.006807   
...                       ...                ...                ...   
20635                1.970414           0.224625           0.001847   
20636                1.957865           0.215208           0.007182   
20637                2.238332           0.215173           0.001688   
20638                2.510121           0.219892           0.002520   
20639                2.007931           0.221185           0.001722   

       ocean_proximity_<1H OCEAN  ocean_proximity_INLAND  \
0                              0                       0   
1                              0                       0   
2                              0                       0   
3                              0                       0   
4                              0                       0   
...                          ...                     ...   
20635                          0                       1   
20636                          0                       1   
20637                          0                       1   
20638                          0                       1   
20639                          0                       1   

       ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
0                           0                         1   
1                           0                         1   
2                           0                         1   
3                           0                         1   
4                           0                         1   
...                       ...                       ...   
20635                       0                         0   
20636                       0                         0   
20637                       0           

In [4]:
with open("Schema.txt", "w") as file:
    [file.write(column + ":" + " Optional[" + str(data[column].dtype) + "]" + "\n") for column in data.columns]

In [9]:
# Let's separate into train and test set
# setting the seed through random_state.

X_train, X_test, y_train, y_test = train_test_split(
    data.drop(['median_ces'], axis=1), # predictive variables
    data['median_ces'], # target
    test_size=0.2, # portion of dataset to allocate to test set
    random_state=0, # we are setting the seed here
)

X_train.shape, X_test.shape

((16512, 17), (4128, 17))

In [10]:
print(data.isnull().sum())

longitude                       0
latitude                        0
housing_median_age              0
total_rooms                     0
total_bedrooms                207
population                      0
households                      0
median_income                   0
median_ces                      0
ocean_proximity                 0
total_rooms_per_person          0
bedrooms_per_room             207
income_per_person               0
ocean_proximity_<1H OCEAN       0
ocean_proximity_INLAND          0
ocean_proximity_ISLAND          0
ocean_proximity_NEAR BAY        0
ocean_proximity_NEAR OCEAN      0
dtype: int64


# Target

In [11]:
y_train = np.log(y_train)
y_test = np.log(y_test)

# Categorical variables

In [12]:
cat_vars = [var for var in data.columns if data[var].dtype == 'O']
print(cat_vars)

['ocean_proximity']


# Numerical variables missing values

In [13]:
# identifying the numerical variables

num_vars = [
    var for var in X_train.columns if var != 'median_ces'
]

# number of numerical variables
len(num_vars)

17

In [14]:
# making a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in num_vars
    if X_train[var].isnull().sum() > 0
]

# print percentage of missing values per variable
X_train[vars_with_na].isnull().mean()

total_bedrooms       0.009569
bedrooms_per_room    0.009569
dtype: float64

In [15]:
# replacing missing values

for var in vars_with_na:

    # calculate the mean using the train set
    mean_val = X_train[var].mean()
    
    print(var, mean_val)

    # add binary missing indicator (in train and test)
    X_train[var + '_na'] = np.where(X_train[var].isnull(), 1, 0)
    X_test[var + '_na'] = np.where(X_test[var].isnull(), 1, 0)

    # replace missing values by the mean
    # (in train and test)
    X_train[var].fillna(mean_val, inplace=True)
    X_test[var].fillna(mean_val, inplace=True)

# check that we have no more missing values in the engineered variables
X_train[vars_with_na].isnull().sum()

total_bedrooms 537.6391096979332
bedrooms_per_room 0.21279519741968828


/var/tmp/ipykernel_3873/1465515309.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train[var].fillna(mean_val, inplace=True)
/var/tmp/ipykernel_3873/1465515309.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)',

total_bedrooms       0
bedrooms_per_room    0
dtype: int64

In [17]:
# re-checking that test set does not contain null values in the engineered variables

[var for var in vars_with_na if X_test[var].isnull().sum() > 0]

[]

In [18]:
# checking the binary missing indicator variables

X_train[['total_bedrooms', 'bedrooms_per_room']].head()

total_bedrooms  bedrooms_per_room
12069            76.0           0.151394
15925           492.0           0.206636
11162           385.0           0.224098
4904            208.0           0.298422
4683            601.0           0.253266

In [19]:
# the yeo-johnson transformation learns the best exponent to transform the variable

x= ["total_rooms", "population", "households"]
for var in x:
    # transform the variable - yeo-johsnon
    X_train[var], param = stats.yeojohnson(X_train[var])

# transformation to the test set with the same
# parameter: see who this time we pass param as argument to the 
# for var in x:
    # transform the variable - yeo-johsnon
    # X_test[var], param = stats.yeojohnson(X_test[var], lmbda=param)
X_test["total_rooms"] = stats.yeojohnson(X_test["total_rooms"], lmbda=param)
X_test["population"] = stats.yeojohnson(X_test["population"], lmbda=param)
X_test["households"] = stats.yeojohnson(X_test["households"], lmbda=param)
print(param)

0.23846706346879348


In [20]:
# check absence of na in the train set
[var for var in X_train.columns if X_train[var].isnull().sum() > 0]

[]

In [21]:
# check absence of na in the test set
[var for var in X_train.columns if X_test[var].isnull().sum() > 0]

[]

# Feature Scaling

In [22]:
X_train.drop('ocean_proximity', axis=1, inplace=True)

In [23]:
X_test.drop('ocean_proximity', axis=1, inplace=True)

In [24]:
# create scaler
scaler = MinMaxScaler()

#  fit  the scaler to the train set
scaler.fit(X_train) 

# transform the train and test set

# sklearn returns numpy arrays, so we wrap the
# array with a pandas dataframe

X_train = pd.DataFrame(
    scaler.transform(X_train),
    columns=X_train.columns
)

X_test = pd.DataFrame(
    scaler.transform(X_test),
    columns=X_train.columns
)

In [25]:
X_train.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0   0.677291  0.137088            0.098039     0.311189        0.011639   
1   0.190239  0.551541            1.000000     0.498354        0.076195   
2   0.632470  0.137088            0.490196     0.453651        0.059590   
3   0.606574  0.156217            0.725490     0.345507        0.032123   
4   0.596614  0.163656            1.000000     0.497877        0.093110   

   population  households  median_income  total_rooms_per_person  \
0    0.215874    0.225651       0.257838                0.039722   
1    0.407729    0.456587       0.268265                0.028884   
2    0.362548    0.428111       0.236783                0.030290   
3    0.327834    0.350665       0.066578                0.016699   
4    0.374837    0.495768       0.184591                0.037711   

   bedrooms_per_room  income_per_person  ocean_proximity_<1H OCEAN  \
0           0.057105           0.016050                        0.0   
1           0.118484           0.002500                        0.0   
2           0.137886           0.003273                        1.0   
3           0.220469           0.001633                        1.0   
4           0.170295           0.002363                        1.0   

   ocean_proximity_INLAND  ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
0                     1.0                     0.0                       0.0   
1                     0.0                     0.0                       1.0   
2                     0.0                     0.0                       0.0   
3                     0.0                     0.0                       0.0   
4                     0.0                     0.0                       0.0   

   ocean_proximity_NEAR OCEAN  total_bedrooms_na  bedrooms_per_room_na  
0                         0.0                0.0                   0.0  
1                         0.0                0.0                   0.0  
2                         0.0                0.0                   0.0  
3                         0.0                0.0                   0.0  
4                         0.0                0.0                   0.0

In [26]:
X_test.head()

longitude  latitude  housing_median_age  total_rooms  total_bedrooms  \
0   0.727092  0.004251            0.411765     0.532512        0.061763   
1   0.635458  0.146652            0.607843     0.568890        0.061608   
2   0.250000  0.649309            0.549020     0.601703        0.084885   
3   0.871514  0.070138            0.705882     0.378974        0.028864   
4   0.191235  0.557917            0.470588     0.493635        0.060987   

   population  households  median_income  total_rooms_per_person  \
0    0.430076    0.429273       0.251852                0.024379   
1    0.406335    0.450457       0.364112                0.036459   
2    0.430337    0.483817       0.265431                0.036755   
3    0.271326    0.306429       0.134564                0.032770   
4    0.323881    0.432339       0.310685                0.045528   

   bedrooms_per_room  income_per_person  ocean_proximity_<1H OCEAN  \
0           0.099900           0.002258                        0.0   
1           0.057676           0.003803                        1.0   
2           0.081149           0.002363                        0.0   
3           0.181946           0.005385                        0.0   
4           0.156316           0.006621                        0.0   

   ocean_proximity_INLAND  ocean_proximity_ISLAND  ocean_proximity_NEAR BAY  \
0                     0.0                     0.0                       0.0   
1                     0.0                     0.0                       0.0   
2                     1.0                     0.0                       0.0   
3                     1.0                     0.0                       0.0   
4                     0.0                     0.0                       1.0   

   ocean_proximity_NEAR OCEAN  total_bedrooms_na  bedrooms_per_room_na  
0                         1.0                0.0                   0.0  
1                         0.0                0.0                   0.0  
2                         0.0                0.0                   0.0  
3                         0.0                0.0                   0.0  
4                         0.0                0.0                   0.0

In [27]:
X_test.dtypes
# cat_vars = [var for var in X_test.columns if data[var].dtype == 'O']

longitude                     float64
latitude                      float64
housing_median_age            float64
total_rooms                   float64
total_bedrooms                float64
population                    float64
households                    float64
median_income                 float64
total_rooms_per_person        float64
bedrooms_per_room             float64
income_per_person             float64
ocean_proximity_<1H OCEAN     float64
ocean_proximity_INLAND        float64
ocean_proximity_ISLAND        float64
ocean_proximity_NEAR BAY      float64
ocean_proximity_NEAR OCEAN    float64
total_bedrooms_na             float64
bedrooms_per_room_na          float64
dtype: object

In [28]:
X_train.to_csv('xtrain.csv', index=False)
X_test.to_csv('xtest.csv', index=False)

y_train.to_csv('ytrain.csv', index=False)
y_test.to_csv('ytest.csv', index=False)

In [29]:
# now let's save the scaler

joblib.dump(scaler, 'minmax_scaler.joblib') 

['minmax_scaler.joblib']